## 1. Imports and definitions

In [2]:
import pandas as pd
import numpy as np

# Modelling and Forecasting
# ==============================================================================
import sklearn
import skforecast
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import bayesian_search_forecaster

# Viz
# ==============================================================================
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
plt.style.use('seaborn-v0_8-darkgrid')


### 1.1. Cyclical functions - doesn't work

In [29]:
# Cliclical encoding of direction features
# ==============================================================================
def cyclical_encoding(data: pd.Series, cycle_length: int) -> pd.DataFrame:
    """
    Encode a cyclical feature with two new features sine and cosine.
    The minimum value of the feature is assumed to be 0. The maximum value
    of the feature is passed as an argument.
      
    Parameters
    ----------
    data : pd.Series
        Series with the feature to encode.
    cycle_length : int
        The length of the cycle. For example, 12 for months, 24 for hours, etc.
        This value is used to calculate the angle of the sin and cos.

    Returns
    -------
    result : pd.DataFrame
        Dataframe with the two new features sin and cos.

    """

    sin = np.sin(2 * np.pi * data/cycle_length)
    cos = np.cos(2 * np.pi * data/cycle_length)
    result =  pd.DataFrame({
                  f"{data.name}_sin": sin,
                  f"{data.name}_cos": cos
              })

    return result


In [30]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

class SineCosineEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cycle_length=360):
        self.cycle_length = cycle_length
        self.transform_storage = {}
        self.last_series_name = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            self.last_series_name = X.columns[0]
            self.series_index = X.index
            X = X.iloc[:, 0]
        elif isinstance(X, pd.Series):
            self.last_series_name = X.name
            self.series_index = X.index
        else:
            raise ValueError("Input must be a pandas DataFrame or Series.")
        
        X = np.asarray(X).ravel()
        sine = np.sin(2 * np.pi * X / self.cycle_length)
        cosine = np.cos(2 * np.pi * X / self.cycle_length)
        
        # Store the cosine components by series name
        self.transform_storage[self.last_series_name] = cosine
        
        return pd.DataFrame({self.last_series_name: sine}, index=self.series_index)

    def inverse_transform(self, X):
        if self.last_series_name not in self.transform_storage:
            raise ValueError("Transformation has not been applied yet for the series.")

        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input must be a pandas DataFrame.")
        
        series_name = X.columns[0]
        if series_name not in self.transform_storage:
            raise ValueError(f"Transformation has not been applied yet for the series '{series_name}'.")

        sine = np.asarray(X[series_name]).ravel()
        cosine = self.transform_storage[series_name]
        
        if len(sine) > len(cosine):
            raise ValueError(f"Shape mismatch: sine length is {len(sine)}, but expected length is {len(cosine)}.")
        
        # Trim cosine to match the length of sine
        cosine = cosine[:len(sine)]
        
        # Calculate angles from sine and cosine
        angles = np.arctan2(sine, cosine)
        return pd.Series(angles * self.cycle_length / (2 * np.pi), index=X.index, name=series_name)

    def get_feature_names_out(self, input_features=None):
        return [self.last_series_name if self.last_series_name else 'sine_component']


## 2. Read and transform data 

In [10]:
# Read data in
data_dir = '../../Data/mooloolaba'
file = '/mool_all.csv'

df = pd.read_csv(data_dir + file)
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index(keys = 'datetime', inplace=True)
df = df.asfreq('30min')

df.head()

,wave_height_significant,wave_height_max,wave_period_upx,wave_period_peak,wave_direction,sea_temperature
datetime,,,,,,
2022-01-01 00:00:00,2.477,4.110,6.136,8.230,90.0,25.75
2022-01-01 00:30:00,2.259,3.240,5.871,8.179,98.0,25.75
2022-01-01 01:00:00,2.268,4.280,5.752,8.174,87.0,25.80
2022-01-01 01:30:00,2.452,3.970,5.987,8.719,80.0,25.80
2022-01-01 02:00:00,2.260,3.514,5.824,8.654,90.0,25.75


In [11]:
# Slice off target variables
target_vars = ['wave_height_max', 'wave_period_upx', 'wave_direction']
df_target = df[target_vars].copy()
df_target.rename(columns={
    'wave_height_max':'wave_height',
    'wave_period_upx':'wave_period',
}, inplace=True)
df_target

,wave_height,wave_period,wave_direction
datetime,,,
2022-01-01 00:00:00,4.110,6.136,90.0
2022-01-01 00:30:00,3.240,5.871,98.0
2022-01-01 01:00:00,4.280,5.752,87.0
2022-01-01 01:30:00,3.970,5.987,80.0
2022-01-01 02:00:00,3.514,5.824,90.0
...,...,...,...
2024-03-31 21:30:00,2.220,4.937,125.0
2024-03-31 22:00:00,2.300,5.370,83.0
2024-03-31 22:30:00,2.150,5.186,85.0


In [12]:
for col in df_target.columns:
    print(df_target[col].isnull().value_counts())

wave_height
False    38646
True       762
Name: count, dtype: int64
wave_period
False    38646
True       762
Name: count, dtype: int64
wave_direction
False    38611
True       797
Name: count, dtype: int64


In [13]:
# Train-test split
# ==============================================================================
one_month = (-1)*30*48 # One month
two_months = (-1)*60*48 # Two months

end_val = two_months
end_train = end_val + two_months 

df_train = df_target.iloc[:end_train].copy()
df_val = df_target.iloc[end_train:end_val].copy()
df_test = df_target.iloc[end_val:].copy()

print(f"Train dates      : {df_train.index.min()} --- {df_train.index.max()}  (n={len(df_train)})")
print(f"Validation dates : {df_val.index.min()} --- {df_val.index.max()}  (n={len(df_val)})")
print(f"Test dates       : {df_test.index.min()} --- {df_test.index.max()}  (n={len(df_test)})")

Train dates      : 2022-01-01 00:00:00 --- 2023-12-02 23:30:00  (n=33648)
Validation dates : 2023-12-03 00:00:00 --- 2024-01-31 23:30:00  (n=2880)
Test dates       : 2024-02-01 00:00:00 --- 2024-03-31 23:30:00  (n=2880)


In [23]:
## Impute missings as -99
df_target = df_target.replace(np.nan, -99.99)
df_target.isnull().sum()

wave_height       0
wave_period       0
wave_direction    0
dtype: int64

In [16]:
print(df_val.isnull().sum())
print(df_test.isnull().sum())

wave_height       5
wave_period       5
wave_direction    9
dtype: int64
wave_height       1
wave_period       1
wave_direction    3
dtype: int64


## 4. Separate models

### 4.1 Wave height

In [24]:
# Hyperparameters search
# ==============================================================================

# Define forecaster
forecaster_height = ForecasterAutoreg(
                regressor = LGBMRegressor(random_state=42),
                lags      = 24
             )

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'  : trial.suggest_int('n_estimators', 600, 1200, step=50),
        'max_depth'     : trial.suggest_int('max_depth', 3, 12, step=1),
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.5),
        'reg_alpha'     : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'    : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'subsample'     : trial.suggest_float('subsample', 0.1, 1, step = 0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1., step = 0.1),
        'lags'          : trial.suggest_categorical('lags', [24, 36, 48])
    } 
    return search_space

results_search, frozen_trial = bayesian_search_forecaster(
                                   forecaster         = forecaster_height,
                                   y                  = df_target.iloc[:end_val]['wave_height'],
                                   steps              = 24,
                                   metric             = 'mean_absolute_error',
                                   search_space       = search_space,
                                   initial_train_size = len(df_target[:end_train]),
                                   refit              = False,
                                   n_trials           = 50, # Increase for more exhaustive search
                                   random_state       = 42,
                                   return_best        = True,
                                   n_jobs             = 'auto',
                                   verbose            = False,
                                   show_progress      = True
                               )

# Backtest final model on test data
# ==============================================================================
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster_height,
                          y                  = df_target['wave_height'],
                          steps              = 24,
                          metric             = 'mean_absolute_error',
                          initial_train_size = len(df_target[:end_val]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False, 
                          show_progress      = True
                      )

print(f"Backtest error (MAE): {metric:.2f}")

  0%|          | 0/50 [00:00<?, ?it/s]

[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001916 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6120
[LightGBM] [Info] Number of data points in the train set: 33624, number of used features: 24
[LightGBM] [Info] Start training from score -0.141838
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM] [Warning] Unknown parameter: sample_weight
[LightGBM

KeyboardInterrupt: 

In [19]:
help(LGBMRegressor)

Help on class LGBMRegressor in module lightgbm.sklearn:

class LGBMRegressor(sklearn.base.RegressorMixin, LGBMModel)
 |  LGBMRegressor(boosting_type: str = 'gbdt', num_leaves: int = 31, max_depth: int = -1, learning_rate: float = 0.1, n_estimators: int = 100, subsample_for_bin: int = 200000, objective: Union[str, Callable[[Optional[numpy.ndarray], numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], Tuple[numpy.ndarray, numpy.ndarray]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = None, class_weight: Union[Dict, str, NoneType] = None, min_split_gain: float = 0.0, min_child_weight: float = 0.001, min_child_samples: int = 20, subsample: float = 1.0, subsample_freq: int = 0, colsample_bytree: float = 1.0, reg_alpha: float = 0.0, reg_lambda: float = 0.0, random_state: Union[int, numpy.random.mtrand.RandomState